In [2]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [4]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [16]:
#Lectura pagos Bradesco
pagos_brad_c0 = pd.read_excel(''+ruta_pagos+'/Bradesco/Facturacion_c0/Facturacion c0.xlsx'dtype={'TARJETA': object})
pagos_brad_c0 = pagos_brad_c0.drop_duplicates()
pagos_brad_c02 = pd.pivot_table(pagos_brad_c0,index=['TARJETA','FECHA','BUCKET','PRODUCTO'],values=['PAGO','COMISION'],aggfunc=np.sum)
pagos_brad_c02 = pd.DataFrame(pagos_brad_c02.to_records())

In [26]:
#Lectura pagos Bradesco
pagos_brad_extra = pd.read_excel(''+ruta_pagos+'/Bradesco/Facturacion_extra/Facturacion extra.xlsx'dtype={'TARJETA': object})
pagos_brad_extra = pagos_brad_extra.drop_duplicates()
pagos_brad_extra2 = pd.pivot_table(pagos_brad_extra,index=['TARJETA','FECHA','BUCKET','PRODUCTO'],values=['PAGO','COMISION'],aggfunc=np.sum)
pagos_brad_extra2 = pd.DataFrame(pagos_brad_extra2.to_records())

In [47]:
#Lectura pagos Liverpool
pagos_liv = pd.read_excel(''+ruta_pagos+'/Liverpool/Pagos/Facturacion Liverpool.xlsx',converters={'Cuenta':str})

,Cuenta,Fecha pago,Fecha Posteo,monto,Descripcion,Division
0,0004178490000277468,02/01/2020,02/01/2020,500.0,5200000000008917,N2


In [76]:
#Lectura pagos Banco Azteca
pagos_baz = pd.read_excel(''+ruta_pagos+'/Banco Azteca/Pagos/Pagos recovery-26.xlsx')

# Bradesco

In [57]:
#Concatenamos los pagos de brad
pagos_brad = pd.concat([pagos_brad_c0,pagos_brad_extra],ignore_index=True)
pagos_brad = pagos_brad.loc[pagos_brad['FECHA'].map(str)!='00:00:00']
pagos_brad['FECHA'] = pd.to_datetime(pagos_brad['FECHA'],dayfirst=False)
pagos_brad['YEAR'] = pagos_brad['FECHA'].dt.year
pagos_brad['MONTH'] = pagos_brad['FECHA'].dt.month
pagos_brad['credito_unico'] = pagos_brad['TARJETA'].map(str)+'-'+pagos_brad['MONTH'].map(str)+'-'+pagos_brad['YEAR'].map(str)
pagos_brad['YEAR_MONTH'] = pagos_brad['YEAR'].map(str)+'-'+pagos_brad['MONTH'].map(str)
pagos_brad = pagos_brad.loc[pagos_brad['YEAR'] >= 2019]
pagos_brad = pagos_brad.iloc[:,[1,3,6,2,9,10,11,12]]
pagos_brad.columns = ['Cuenta','Fecha pago','Division','monto','YEAR','MONTH','credito_unico','YEAR_MONTH']
pagos_brad['Cliente'] = 'Bradesco' 
pagos_brad.head(1)

,Cuenta,Fecha pago,Division,monto,YEAR,MONTH,credito_unico,YEAR_MONTH,Cliente
0,2869002607825000,2020-07-30,5,1542.0,2020,7,2869002607825000-7-2020,2020-7,Bradesco


# Liverpool

In [58]:
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Cuenta','Fecha pago','Division'],values=['monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
pagos_liv2['Fecha pago'] = pd.to_datetime(pagos_liv2['Fecha pago'],dayfirst=True)
pagos_liv2['YEAR'] = pagos_liv2['Fecha pago'].dt.year
pagos_liv2['MONTH'] = pagos_liv2['Fecha pago'].dt.month
pagos_liv2['credito_unico'] = pagos_liv2['Cuenta'].map(str)+'-'+pagos_liv2['MONTH'].map(str)+'-'+pagos_liv2['YEAR'].map(str)
pagos_liv2['YEAR_MONTH'] = pagos_liv2['YEAR'].map(str)+'-'+pagos_liv2['MONTH'].map(str)
pagos_liv2 = pagos_liv2.loc[pagos_liv2['YEAR'] >= 2019]
pagos_liv2['Cliente'] = 'Liverpool'
pagos_liv2.head(1)

,Cuenta,Fecha pago,Division,monto,YEAR,MONTH,credito_unico,YEAR_MONTH,Cliente
0,0000010000003513825,2020-07-24,COF,500.0,2020,7,0000010000003513825-7-2020,2020-7,Liverpool


In [59]:
#Hacemos la union entre promesas y pagos
pagos_brad_liv = pd.concat([pagos_brad,pagos_liv2],ignore_index=True)

In [68]:
total_pagos_mes = pd.pivot_table(pagos_brad_liv,index=['Cliente'],columns=['YEAR_MONTH'],values=['monto'],aggfunc=np.sum, margins=True)
total_pagos_mes = pd.DataFrame(total_pagos_mes.to_records())
total_pagos_mes = total_pagos_mes.iloc[:,:-1]
total_pagos_mes['Promedio'] = total_pagos_mes.mean(numeric_only=True,axis=1).round(2)
total_pagos_mes

,Cliente,"('monto', '2019-1')","('monto', '2019-10')","('monto', '2019-11')","('monto', '2019-12')","('monto', '2019-2')","('monto', '2019-3')","('monto', '2019-4')","('monto', '2019-5')","('monto', '2019-6')",...,"('monto', '2020-11')","('monto', '2020-2')","('monto', '2020-3')","('monto', '2020-4')","('monto', '2020-5')","('monto', '2020-6')","('monto', '2020-7')","('monto', '2020-8')","('monto', '2020-9')",Promedio
0,Bradesco,3678102.19,5426913.2,5496582.19,6649977.42,4663196.31,6144176.84,4410956.03,5972336.53,4990561.33,...,4899667.03,4215084.14,4054285.78,2257007.20,2677542.03,4708089.72,7331414.66,7901981.89,7505988.99,5249906.39
1,Liverpool,NaN,NaN,NaN,28599.00,NaN,NaN,NaN,NaN,NaN,...,161837.00,2872860.10,2780681.54,3032286.74,3130591.87,3618743.56,3947593.95,3594131.05,2920522.56,2650922.16
2,All,3678102.19,5426913.2,5496582.19,6678576.42,4663196.31,6144176.84,4410956.03,5972336.53,4990561.33,...,5061504.03,7087944.24,6834967.32,5289293.94,5808133.90,8326833.28,11279008.61,11496112.94,10426511.55,6632996.21


# Banco Azteca

In [77]:
pagos_baz2 = pagos_baz.drop_duplicates()
pagos_baz2['Mes'] = pagos_baz2['Inicio de Semana'].dt.month
pagos_baz2.head(1)

,Año,Número Semana,Inicio de Semana,Final de Semana,JEFATURA DE COBRANZA,REGIONAL,GERENTE,Folio de Facturación,Total Recuperado,Total Comisión,Estatus,Fecha de Generación,Cartera,Mes
0,2020,51,2020-12-14,2020-12-20,8693-VERACRUZ / CARDEL,9164 GCC VERACRUZ CENTRO,TAPIA CARO JOSE LUCINO,20514497491641,1754.0,434.0,GENERADA,2020-12-21,Recovery,12


In [88]:
total_pagos_mes_baz = pd.pivot_table(pagos_baz2,index=['Cartera'],columns=['Mes'],values=['Total Comisión'],aggfunc=np.sum,margins=True)
total_pagos_mes_baz = pd.DataFrame(total_pagos_mes_baz.to_records())
total_pagos_mes_baz = total_pagos_mes_baz.iloc[:,:-2]
total_pagos_mes_baz['Promedio'] = total_pagos_mes_baz.mean(numeric_only=True,axis=1).round(2)
total_pagos_mes_baz

,Cartera,"('Total Comisión', 1)","('Total Comisión', 2)","('Total Comisión', 3)","('Total Comisión', 4)","('Total Comisión', 5)","('Total Comisión', 6)","('Total Comisión', 7)","('Total Comisión', 8)","('Total Comisión', 9)","('Total Comisión', 10)","('Total Comisión', 11)",Promedio
0,26-39,130626.02,212267.99,265012.24,144546.31,275292.94,418912.34,276712.17,303359.67,281539.94,277916.11,315719.10,263809.53
1,Recovery,570062.93,1054812.49,1147021.12,624110.72,680143.21,1042809.13,791691.99,833067.05,418109.59,454287.66,411255.84,729761.07
2,All,700688.95,1267080.48,1412033.36,768657.03,955436.15,1461721.47,1068404.16,1136426.72,699649.53,732203.77,726974.94,993570.60


In [90]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Pagos historicos Naucalpan.xlsx',engine='xlsxwriter')
total_pagos_mes.to_excel(writer,'BradLiv',index=False,header=True)
total_pagos_mes_baz.to_excel(writer,'BancoAzteca',index=False,header=True)


writer.save()